In [1]:
import re
import pdfplumber
import pandas as pd
from pprint import pprint as pp
from geopy.distance import distance
from geopy.geocoders import Nominatim
import sqlite3
import csv

In [2]:
pdf_path = r'C:\Users\Roshi\Documents\Git_Repos\la_tax_auction_parse\data\2025B-Auction-Book.pdf'
csv_path = r'C:\Users\Roshi\Documents\Git_Repos\la_tax_auction_parse\data\auction_enhanced.csv'
tables = []
df = pd.DataFrame()
# Setup geopy
geolocator = Nominatim(user_agent="assessor_parse")

In [3]:
with pdfplumber.open(pdf_path) as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        table_data = page.extract_table({
            "vertical_strategy": "lines", 
            "horizontal_strategy": "lines",
        })

        if table_data:
            clean_data = [x for x in table_data if x.count('') != len(x)]
            if len(clean_data) > 1:
                df_page = pd.DataFrame(clean_data[1:], columns=clean_data[0])
                tables.append(df_page)

if tables:
    df = pd.concat(tables, ignore_index=True)

In [4]:
def clean_address(addr):
    # Remove newlines
    addr = addr.replace("\n", " ")
    # Collapse multiple spaces
    addr = re.sub(r"\s+", " ", addr)
    # Remove trailing 4-digit extension after ZIP (e.g. "91303 1334" → "91303")
    addr = re.sub(r"(\d{5})\s\d{4}$", r"\1", addr)
    # Trim spaces
    return addr.strip()

def get_lat_long(addr):
    if pd.isna(addr) or addr.strip().upper() == "VACANT LOT":
        return None, None   # skip these
    try:
        location = geolocator.geocode(addr)
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Error geocoding {addr}: {e}")
    return None, None

In [12]:
# Apply function to new column
df["clean_address"] = df["PROPERTY ADDRESS"].apply(clean_address)

# Apply function → returns tuple
df[["Latitude", "Longitude"]] = df["clean_address"].apply(
    lambda x: pd.Series(get_lat_long(x))
)

Error geocoding 7348 JORDAN AVE LOS ANGELES CA 91303: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=7348+JORDAN+AVE+LOS+ANGELES+CA+91303&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error geocoding 7308 CORBIN AVE UNIT F LOS ANGELES CA 91335: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=7308+CORBIN+AVE+UNIT+F+LOS+ANGELES+CA+91335&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error geocoding 18415 COLLINS ST UNIT C LOS ANGELES CA 91356: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=18415+COLLINS+ST+UNIT+C+LOS+ANGELES+CA+91356&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(

In [3]:
with open(csv_path, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    data = list(reader)

# pp(data[:3])  # preview first 3 rows

In [4]:
import re

counter = 0
for row in data:
    loc = row.get('Location_1', '')
    counter += 1

    if loc:
        match = re.search(r"\(([^,]+),\s*([^)]+)\)", loc)
        if match:
            lat, lon = match.groups()
            try:
                row['Latitude'] = float(lat)
                row['Longitude'] = float(lon)
            except ValueError:
                print("   -> Skipped: could not convert to float")


   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert to float
   -> Skipped: could not convert t

In [5]:
test_df = pd.DataFrame(data)

dedupe_keys = [
    "ITEM", "AIN", "MIN BID", "IMP", "NSB#",
    "LEGAL DESCRIPTION", "LOCATION", "PROPERTY ADDRESS", "clean_address"
]

# ensure RollYear is numeric
test_df["RollYear"] = pd.to_numeric(test_df["RollYear"], errors="coerce")

# keep only the row with max RollYear for each group
test_df_deduped = test_df.loc[test_df.groupby(dedupe_keys)["RollYear"].idxmax()]

# back to list-of-dicts if needed
deduped_data = test_df_deduped.to_dict(orient="records")


In [7]:
print(len(deduped_data))
pp(deduped_data[0])

1260
{'AIN': '2017002036',
 'AdministrativeRegion': '02',
 'AssessorID': '2017-002-036',
 'Bathrooms': '0',
 'Bedrooms': '0',
 'CENTER_LAT': '34.231051389999998',
 'CENTER_LON': '-118.64824919',
 'City': '',
 'Cluster': '02121',
 'EffectiveYearBuilt': '0',
 'FixtureExemption': '0.00',
 'FixtureValue': '0.00',
 'GeneralUseType': 'Residential',
 'HomeownersExemption': '0.00',
 'HouseFraction': '',
 'HouseNo': '0',
 'IMP': '',
 'ITEM': '1472',
 'ImpBaseYear': '1998',
 'ImprovementValue': '0.00',
 'LEGAL DESCRIPTION': 'TRACT # 9683 LOT 22',
 'LOCATION': 'CITY-LOS ANGELES',
 'LandBaseYear': '1998',
 'LandValue': '6920.00',
 'Latitude': 34.23105139,
 'Location_1': '(34.23105139, -118.64824919)',
 'Longitude': -118.64824919,
 'MIN BID': '$9,459',
 'NSB#': '536',
 'PROPERTY ADDRESS': 'VACANT LOT',
 'ParcelBoundaryDescription': 'TRACT # 9683 LOT   22',
 'PersonalPropertyExemption': '0.00',
 'PersonalPropertyValue': '0.00',
 'PropertyLocation': '',
 'PropertyType': 'VAC',
 'PropertyUseCode': '01

In [8]:
current = (33.76403641974573, -118.20115231391159)

for loc in deduped_data:
    if loc['Latitude'] and loc['Longitude']:
        # print(loc['Latitude'], loc['Longitude'])
        loc_coord = (float(loc['Latitude']), float(loc['Longitude']))
        loc['distance'] = distance(current, loc_coord).miles
    
# sort by distance (smallest first)
data_sorted = sorted(
    [d for d in deduped_data if 'distance' in d],
    key=lambda x: x['distance']
)

In [9]:
pp(data_sorted)

[{'AIN': '7281010040',
  'AdministrativeRegion': '10',
  'AssessorID': '7281-010-040',
  'Bathrooms': '1',
  'Bedrooms': '2',
  'CENTER_LAT': '33.771193340000004',
  'CENTER_LON': '-118.18675801000001',
  'City': 'LONG BEACH CA',
  'Cluster': '10500',
  'EffectiveYearBuilt': '1956',
  'FixtureExemption': '0.00',
  'FixtureValue': '0.00',
  'GeneralUseType': 'Residential',
  'HomeownersExemption': '7000.00',
  'HouseFraction': '',
  'HouseNo': '333',
  'IMP': 'Y',
  'ITEM': '3305',
  'ImpBaseYear': '1991',
  'ImprovementValue': '49974.00',
  'LEGAL DESCRIPTION': 'LONG BEACH JEANEANE APTS OYO APT 6 G-5 N',
  'LOCATION': 'CITY-LONG BEACH',
  'LandBaseYear': '1991',
  'LandValue': '58306.00',
  'Latitude': 33.77119334,
  'Location_1': '(33.77119334, -118.18675801)',
  'Longitude': -118.18675801,
  'MIN BID': '$12,842',
  'NSB#': '509',
  'PROPERTY ADDRESS': '333 LINDEN AVE 6 LONG BEACH CA 90802 2511',
  'ParcelBoundaryDescription': 'LONG BEACH JEANEANE APTS OYO APT 6 G-5 N',
  'PersonalPro

In [10]:
final_df = pd.DataFrame(data_sorted)

In [16]:
final_df[['AssessorID','Bathrooms','Bedrooms', 'RollYear','TotalValue', 'SQFTmain', 'MIN BID', 'PROPERTY ADDRESS', 'Latitude', 'Longitude', 'distance', 'SpecificUseType', 'SpecificUseDetail1', 'SpecificUseDetail2']].head(50)

,AssessorID,Bathrooms,Bedrooms,RollYear,TotalValue,SQFTmain,MIN BID,PROPERTY ADDRESS,Latitude,Longitude,distance,SpecificUseType,SpecificUseDetail1,SpecificUseDetail2
0,7281-010-040,1,2,2021,108280.00,903,"$12,842",333 LINDEN AVE 6 LONG BEACH CA 90802 2511,33.771193,-118.186758,0.964266,Single Family Residence,,Own-your-own
1,7273-027-052,1,0,2021,53255.00,360,"$8,752",635 ELM AVE 16 LONG BEACH CA 90802 1417,33.774890,-118.188056,1.061990,Single Family Residence,,Own-your-own
2,7281-012-116,1,0,2021,66609.00,440,"$13,309",723 E 3RD STREET 17+ LONG BEACH CA 90802 0000,33.770824,-118.183163,1.136268,Single Family Residence,,Own-your-own
3,7273-011-065,3,2,2021,221866.00,1083,"$50,110",941 ELM AVE NO 16 LONG BEACH CA 90813 4458,33.778722,-118.188044,1.262429,Single Family Residence,,Condominium
4,7269-013-012,2,2,2021,141299.00,1472,"$33,675",134 E 15TH ST LONG BEACH CA 90813 1802,33.785171,-118.191580,1.557358,Single Family Residence,,
5,7263-007-028,1,2,2021,494323.00,1382,"$6,444",252 CHERRY AVE LONG BEACH CA 90802 3929,33.768066,-118.167555,1.953774,Single Family Residence,,
6,7429-023-036,0,0,2021,227331.00,0,"$3,773",2115 W 16TH ST LONG BEACH CA 90813 1009,33.787555,-118.220188,1.956472,Industrial,,Vacant Land
7,7268-025-035,3,6,2021,568820.00,3796,"$28,206",1736 MARTIN LUTHER KING LONG BEACH CA 90813\n0000,33.788744,-118.180342,2.081902,Three Units (Any Combination),4 Stories or Less,
8,7268-025-052,0,0,2021,157635.00,0,"$18,055",1720 LEWIS AVE LONG BEACH CA 90813 2120,33.788469,-118.179298,2.101824,Single Family Residence,,Vacant Land
9,7262-024-021,11,11,2021,1791274.00,6516,"$23,211",2009 E 7TH ST LONG BEACH CA 90804 4590,33.775650,-118.167341,2.104366,Store Combination,Store and Residential Combination,One Story
